## CKA analysis attempt of the same files between different models

In [1]:
import pickle
import torch
import os

In [2]:
mappings = {
    "yatc": lambda x: x.split("/")[-1].removesuffix(".png"),
    "etbert": lambda x: x.split("/")[-1].removesuffix(".pcap\n"),
    "netmamba": lambda x: x.split("/")[-1].removesuffix(".png"),
    "netfound": lambda x: x.split("/")[-1].removesuffix(".pcap.6").removesuffix(".pcap.17").removesuffix(".pcap.1"),
}

def map_filenames(filepath, filename_mapping):
    with open(filepath, "rb") as f:
        embeddings, filenames = pickle.load(f)
        if embeddings.shape[0] != len(filenames):
            print(filepath)
            

    return embeddings, [filename_mapping(x) for x in filenames]

In [3]:
datasets = {"cross", "caida", "cicids", "cicapt", "mawi"}
models = {"yatc", "etbert", "netfound", "netmamba"}

def dataset_name(filepath):
    for dataset in datasets:
        if dataset in filepath:
            return dataset
    return None

result = {
    key: {k: None for k in models}
    for key in datasets
}

In [4]:
for model in models:
    directory = f'../data/{model}/'
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        dataset = dataset_name(filepath)
        if dataset is None:
            continue
        result[dataset][model] = map_filenames(filepath, mappings[model])

In [5]:
# verification that filenames are preprocessed in the correct way - the intersection of filenames between models should not be much less than a number of files of any of them
for dataset in datasets:
    filenames = [set(result[dataset][model][1]) for model in models]
    print(dataset, len(filenames[0]), len(set.intersection(*filenames)))

caida 1351680 989078
cross 44544 27232
cicapt 1576960 1232708
cicids 557056 435632
mawi 997376 698286


In [6]:
def preprocess_embeddings(result, models, datasets):
    aligned_embeddings = {dataset: {} for dataset in datasets}

    for dataset in datasets:
        # Compute intersection of filenames across all models
        filename_sets = [set(result[dataset][model][1]) for model in models]
        common_filenames = set.intersection(*filename_sets)
        
        # Create a consistent ordering for common filenames
        sorted_common_filenames = sorted(common_filenames)

        for model in models:
            embeddings, filenames = result[dataset][model]

            # Map filenames to indices for fast lookup
            filename_to_index = {filename: idx for idx, filename in enumerate(filenames)}

            # Check if embeddings size matches the filenames length
            assert embeddings.shape[0] == len(filenames), \
                f"Embeddings size ({embeddings.shape[0]}) and filenames length ({len(filenames)}) mismatch for model {model}"

            # Filter out filenames that might have been incorrectly included
            aligned_indices = [filename_to_index[fn] for fn in sorted_common_filenames if fn in filename_to_index]

            # Convert to tensor for indexing
            aligned_indices_tensor = torch.tensor(aligned_indices, dtype=torch.long)

            # Select embeddings safely using aligned indices
            aligned_tensor = embeddings[aligned_indices_tensor]

            # Store aligned tensor
            aligned_embeddings[dataset][model] = aligned_tensor

    return aligned_embeddings


In [7]:
aligned_embeddings = preprocess_embeddings(result, models, datasets)

KeyboardInterrupt: 

In [ ]:
import numpy as np

def gram_linear(x):
    """Compute Gram (kernel) matrix for a linear kernel.

    Args:
        x: A num_examples x num_features matrix of features.

    Returns:
        A num_examples x num_examples Gram matrix of examples.
    """
    return x.dot(x.T)


def gram_rbf(x, threshold=1.0):
    """Compute Gram (kernel) matrix for an RBF kernel.

    Args:
        x: A num_examples x num_features matrix of features.
        threshold: Fraction of median Euclidean distance to use as RBF kernel
        bandwidth. (This is the heuristic we use in the paper. There are other
        possible ways to set the bandwidth; we didn't try them.)

    Returns:
        A num_examples x num_examples Gram matrix of examples.
    """
    dot_products = x.dot(x.T)
    sq_norms = np.diag(dot_products)
    sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]
    sq_median_distance = np.median(sq_distances)
    return np.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))


def center_gram(gram, unbiased=False):
    """Center a symmetric Gram matrix.

    This is equvialent to centering the (possibly infinite-dimensional) features
    induced by the kernel before computing the Gram matrix.

    Args:
        gram: A num_examples x num_examples symmetric matrix.
        unbiased: Whether to adjust the Gram matrix in order to compute an unbiased
        estimate of HSIC. Note that this estimator may be negative.

    Returns:
        A symmetric matrix with centered columns and rows.
    """
    if not np.allclose(gram, gram.T):
        raise ValueError('Input must be a symmetric matrix.')
    gram = gram.copy()

    if unbiased:
        # This formulation of the U-statistic, from Szekely, G. J., & Rizzo, M.
        # L. (2014). Partial distance correlation with methods for dissimilarities.
        # The Annals of Statistics, 42(6), 2382-2412, seems to be more numerically
        # stable than the alternative from Song et al. (2007).
        n = gram.shape[0]
        np.fill_diagonal(gram, 0)
        means = np.sum(gram, 0, dtype=np.float64) / (n - 2)
        means -= np.sum(means) / (2 * (n - 1))
        gram -= means[:, None]
        gram -= means[None, :]
        np.fill_diagonal(gram, 0)
    else:
        means = np.mean(gram, 0, dtype=np.float64)
        means -= np.mean(means) / 2
        gram -= means[:, None]
        gram -= means[None, :]

    return gram


def cka(gram_x, gram_y, debiased=False):
    """Compute CKA.

    Args:
        gram_x: A num_examples x num_examples Gram matrix.
        gram_y: A num_examples x num_examples Gram matrix.
        debiased: Use unbiased estimator of HSIC. CKA may still be biased.

    Returns:
        The value of CKA between X and Y.
    """
    gram_x = center_gram(gram_x, unbiased=debiased)
    gram_y = center_gram(gram_y, unbiased=debiased)

    # Note: To obtain HSIC, this should be divided by (n-1)**2 (biased variant) or
    # n*(n-3) (unbiased variant), but this cancels for CKA.
    scaled_hsic = gram_x.ravel().dot(gram_y.ravel())

    normalization_x = np.linalg.norm(gram_x)
    normalization_y = np.linalg.norm(gram_y)
    return scaled_hsic / (normalization_x * normalization_y)


def _debiased_dot_product_similarity_helper(
    xty, sum_squared_rows_x, sum_squared_rows_y, squared_norm_x, squared_norm_y,
    n):
  """Helper for computing debiased dot product similarity (i.e. linear HSIC)."""
  # This formula can be derived by manipulating the unbiased estimator from
  # Song et al. (2007).
  return (
      xty - n / (n - 2.) * sum_squared_rows_x.dot(sum_squared_rows_y)
      + squared_norm_x * squared_norm_y / ((n - 1) * (n - 2)))


def feature_space_linear_cka(features_x, features_y, debiased=False):
    """Compute CKA with a linear kernel, in feature space.

    This is typically faster than computing the Gram matrix when there are fewer
    features than examples.

    Args:
        features_x: A num_examples x num_features matrix of features.
        features_y: A num_examples x num_features matrix of features.
        debiased: Use unbiased estimator of dot product similarity. CKA may still be
        biased. Note that this estimator may be negative.

    Returns:
        The value of CKA between X and Y.
    """
    features_x = features_x - np.mean(features_x, 0, keepdims=True)
    features_y = features_y - np.mean(features_y, 0, keepdims=True)

    dot_product_similarity = np.linalg.norm(features_x.T.dot(features_y)) ** 2
    normalization_x = np.linalg.norm(features_x.T.dot(features_x))
    normalization_y = np.linalg.norm(features_y.T.dot(features_y))

    if debiased:
        n = features_x.shape[0]
        # Equivalent to np.sum(features_x ** 2, 1) but avoids an intermediate array.
        sum_squared_rows_x = np.einsum('ij,ij->i', features_x, features_x)
        sum_squared_rows_y = np.einsum('ij,ij->i', features_y, features_y)
        squared_norm_x = np.sum(sum_squared_rows_x)
        squared_norm_y = np.sum(sum_squared_rows_y)

        dot_product_similarity = _debiased_dot_product_similarity_helper(
            dot_product_similarity, sum_squared_rows_x, sum_squared_rows_y,
            squared_norm_x, squared_norm_y, n)
        normalization_x = np.sqrt(_debiased_dot_product_similarity_helper(
            normalization_x ** 2, sum_squared_rows_x, sum_squared_rows_x,
            squared_norm_x, squared_norm_x, n))
        normalization_y = np.sqrt(_debiased_dot_product_similarity_helper(
            normalization_y ** 2, sum_squared_rows_y, sum_squared_rows_y,
            squared_norm_y, squared_norm_y, n))

    return dot_product_similarity / (normalization_x * normalization_y)

In [ ]:
import itertools

for dataset in datasets:
    print(dataset)
    embeddings = aligned_embeddings[dataset]
    
    unique_pairs = list(itertools.combinations(models, 2))
    
    for model1, model2 in unique_pairs:
        cka_from_features = feature_space_linear_cka(embeddings[model1].numpy(), embeddings[model2].numpy(), debiased=False)
        print(f'Linear CKA between models {model1} and {model2}: {cka_from_features}')
    print('-' * 100)

caida
Linear CKA between models yatc and netmamba: 0.3365003673400656
Linear CKA between models yatc and etbert: 0.365893825228769
Linear CKA between models yatc and netfound: 0.2653822777484194
Linear CKA between models netmamba and etbert: 0.43889288062224685
Linear CKA between models netmamba and netfound: 0.6403622709121128
Linear CKA between models etbert and netfound: 0.3841278084302739
----------------------------------------------------------------------------------------------------
cicapt
Linear CKA between models yatc and netmamba: 0.0008857271169928451
Linear CKA between models yatc and etbert: 0.0014339914055425346
Linear CKA between models yatc and netfound: 0.0036548199156202785
Linear CKA between models netmamba and etbert: 0.002344266534687644
Linear CKA between models netmamba and netfound: 0.05766893786832997
Linear CKA between models etbert and netfound: 0.015383305018852851
--------------------------------------------------------------------------------------------